## Import all required libraries

In [1]:
from _get_frames import extract_frames_moviepy
import cv2
# from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os, json
import torch
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
# from _get_answer import answer

## Extract video frames

In [2]:
video_path = "../DatasetDivided/1-EasyCategory/EC_v_11.mp4"
output_frames_path = "video_frames"

In [3]:
extract_frames_moviepy(video_path, output_frames_path)

iteration 0 ,  Frame 0 saved
iteration 1 ,  Frame 10 saved
iteration 2 ,  Frame 20 saved
iteration 3 ,  Frame 30 saved
iteration 4 ,  Frame 40 saved
iteration 5 ,  Frame 50 saved
iteration 6 ,  Frame 60 saved
iteration 7 ,  Frame 70 saved
iteration 8 ,  Frame 80 saved
iteration 9 ,  Frame 90 saved
iteration 10 ,  Frame 100 saved
iteration 11 ,  Frame 110 saved
iteration 12 ,  Frame 120 saved
iteration 13 ,  Frame 130 saved
iteration 14 ,  Frame 140 saved
iteration 15 ,  Frame 150 saved
iteration 16 ,  Frame 160 saved
iteration 17 ,  Frame 170 saved
iteration 18 ,  Frame 180 saved
iteration 19 ,  Frame 190 saved
iteration 20 ,  Frame 200 saved
iteration 21 ,  Frame 210 saved
iteration 22 ,  Frame 220 saved
iteration 23 ,  Frame 230 saved
iteration 24 ,  Frame 240 saved
iteration 25 ,  Frame 250 saved
iteration 26 ,  Frame 260 saved
iteration 27 ,  Frame 270 saved
iteration 28 ,  Frame 280 saved
iteration 29 ,  Frame 290 saved
iteration 30 ,  Frame 300 saved
iteration 31 ,  Frame 310 sav

In [4]:
# List frames in the output folder
frames = os.listdir(output_frames_path)
frames[:6]

['frame_0.jpg',
 'frame_10.jpg',
 'frame_100.jpg',
 'frame_1000.jpg',
 'frame_1010.jpg',
 'frame_1020.jpg']

In [5]:
# Sort frames by the numeric part of the filenames
sorted_frames = sorted(frames, key=lambda x: int(x.split('_')[1].split('.')[0]))

# Example: Print the first 4 sorted frames
print(sorted_frames[0:6])

['frame_0.jpg', 'frame_10.jpg', 'frame_20.jpg', 'frame_30.jpg', 'frame_40.jpg', 'frame_50.jpg']


## Use a pre-trained captioning model

In [6]:
# blip_local_directory = "./huggingface_models/model-blip-image-captioning"
blip2_local_directory = "./huggingface_models/model-blip2-opt-captioning"
model_name = "Salesforce/blip2-opt-2.7b"
# model_name = "Salesforce/blip2-opt-6.7b"
# model_name = "Salesforce/blip2-opt"

In [7]:
# blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base", cache_dir=blip_local_directory)
# blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", cache_dir=blip_local_directory)

blip2_processor = Blip2Processor.from_pretrained(model_name, 
                                                 cache_dir=blip2_local_directory)
blip2_model = Blip2ForConditionalGeneration.from_pretrained(model_name, 
                                                            cache_dir=blip2_local_directory, 
                                                            torch_dtype=torch.float16, 
                                                            low_cpu_mem_usage=True)
blip2_model.to("cuda:0")

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

...

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like Salesforce/blip2-opt-2.7b is not the path to a directory containing a file named model-00002-of-00002.safetensors.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [8]:
bart_local_directory = "./huggingface_models/bart_large_cnn_model"

In [9]:
# Load the model and tokenizer from the local directory
bart_model = BartForConditionalGeneration.from_pretrained(bart_local_directory)
bart_tokenizer = BartTokenizer.from_pretrained(bart_local_directory)

In [ ]:
min_length = 24
max_length = 72
beam_amount = 5

In [ ]:
def caption_image(image_path, model=blip2_model, processor=blip2_processor):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to("cuda:0", torch.float16)
    output = model.generate(**inputs, num_beams=beam_amount, min_length=min_length, max_length=max_length)
    caption = processor.batch_decode(output, skip_special_tokens=True)[0].strip()
    return caption

In [ ]:
image_path = output_frames_path + "/"
cap = caption_image(image_path)
print(cap)

In [15]:
def summarize_all_cap(model, tokenizer, prompt):
    
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
    summary = summarizer(prompt, max_length=1024, min_length=10, do_sample=False)

    return summary[0]['summary_text']

In [19]:
def caption_video(video_path, 
                  output_frames_path="video_frames", 
                  interval=5):

    all_cap = []
    
    # Extract all frames
    extract_frames_moviepy(video_path, output_frames_path)

    # Get & Sort all frames
    frames = os.listdir(output_frames_path)
    sorted_frames = sorted(frames, key=lambda x: int(x.split('_')[1].split('.')[0]))

    for frame in sorted_frames[0:-1:interval]:
        image_path = output_frames_path + '/' + frame
        cap = caption_image(image_path)
        all_cap.append(cap)

    # Combine all captions into a single text
    # all_cap_str = '\n------------------------------------\n'.join(all_cap)
    all_cap_str = " ".join(all_cap)

    # Generate a summary
    # Tell me only what happen in the video
    # summary_prompt = "the following are seperate descriptions of successive frames of a video" + \
    # "I want you to take all of these descriptions and return a reasonable summary for the corresponding video." + \
    # "I want you to write this summary such as a human writer." + \
    # "\n Here are the seperate descriptions : \n " + all_cap_str

    # summary = summarize_all_cap(model=bart_model, tokenizer=bart_tokenizer, prompt=all_cap_str)
    
    # summary = answer(summary_prompt)
    
    # return summary
    return all_cap

###  Summarize the Captions to Get a Video Caption

In [21]:
# video_path = ""
video_path = "../DatasetDivided/1-EasyCategory/EC_v_11.mp4"
# video_caption = caption_video(video_path)
captions = caption_video(video_path)
# print(video_caption)
print(captions)

iteration 0 ,  Frame 0 saved
iteration 1 ,  Frame 10 saved
iteration 2 ,  Frame 20 saved
iteration 3 ,  Frame 30 saved
iteration 4 ,  Frame 40 saved
iteration 5 ,  Frame 50 saved
iteration 6 ,  Frame 60 saved
iteration 7 ,  Frame 70 saved
iteration 8 ,  Frame 80 saved
iteration 9 ,  Frame 90 saved
iteration 10 ,  Frame 100 saved
iteration 11 ,  Frame 110 saved
iteration 12 ,  Frame 120 saved
iteration 13 ,  Frame 130 saved
iteration 14 ,  Frame 140 saved
iteration 15 ,  Frame 150 saved
iteration 16 ,  Frame 160 saved
iteration 17 ,  Frame 170 saved
iteration 18 ,  Frame 180 saved
iteration 19 ,  Frame 190 saved
iteration 20 ,  Frame 200 saved
iteration 21 ,  Frame 210 saved
iteration 22 ,  Frame 220 saved
iteration 23 ,  Frame 230 saved
iteration 24 ,  Frame 240 saved
iteration 25 ,  Frame 250 saved
iteration 26 ,  Frame 260 saved
iteration 27 ,  Frame 270 saved
iteration 28 ,  Frame 280 saved
iteration 29 ,  Frame 290 saved
iteration 30 ,  Frame 300 saved
iteration 31 ,  Frame 310 sav

In [23]:
liste_sans_doublons = list(dict.fromkeys(captions))
print(liste_sans_doublons)

['a man sleeping on a bench in the park', 'a man in a suit and tie standing in front of trees', 'a man in a suit and tie standing in front of a tree', 'a man is sleeping on a bench in the park', 'a man is sitting on the ground with his head down', 'a man sitting on the ground in front of a tree', 'a man in a suit holding a bag of food', 'a man with a green jacket and a green jacket', 'a man with a beard and a green jacket', 'a man in a suit holding up a bunch of money', 'a man in a suit and tie is talking to another man in a suit', 'a man in a suit holding a piece of paper', 'a man in a suit and tie holding a cigarette', 'a person holding a bag of money and a plastic bag of coins', 'a person holding a bag of food in their hands', 'a man sitting on a bench in the park', 'a person holding a coin in their hand', 'a man in a green jacket is holding a cell', 'a man sitting on a bench eating food', 'a man sitting on a bench with a book', 'a person is putting tickets to someone', 'a person ho

### Labeling all videos in this dataset

#### 1- Easy Category

In [36]:
category_name = "1-EasyCategory"
category_path = "../DatasetDivided" + "/" + category_name

In [ ]:
videos = os.listdir(category_path)
print(videos)
print(len(videos))

['EC_v_1.mp4', 'EC_v_10.mp4', 'EC_v_11.mp4', 'EC_v_2.mp4', 'EC_v_3.mp4', 'EC_v_4.mp4', 'EC_v_5.mp4', 'EC_v_6.mp4', 'EC_v_7.mp4', 'EC_v_8.mp4', 'EC_v_9.mp4']
11


In [ ]:
if not os.path.exists("labels"):
    os.makedirs("labels")
# Save the updated list to a JSON file
output_file_labels = "labels/ec_videos_labels.json"

In [ ]:
videos_summaries = []

for video in videos:
    path = category_path + "/" + video
    cap_vid = caption_video(path)
    videos_summaries.append(
        {
            "video_category": category_name,
            "video_name": video,
            "summary": cap_vid
        }
    )

with open(output_file_labels, "w") as file:
    json.dump(videos_summaries, file, indent=4)

#### 2- Medium Category

In [ ]:
category_name = "2-MediumCategory"
category_path = "../DatasetDivided" + "/" + category_name

In [ ]:
sub_cat_mc_path = os.listdir(category_path)
print(sub_cat_mc_path)

In [ ]:
if not os.path.exists("labels"):
    os.makedirs("labels")
# Save the updated list to a JSON file
output_file_labels = "labels/mc_videos_labels.json"

In [ ]:
videos_summaries = []
for sub_cat in sub_cat_mc_path:
    sub_cat_path = category_path + '/' + sub_cat
    videos = os.listdir(sub_cat_path)
    for video in videos:
        path = sub_cat_path + "/" + video
        cap_vid = caption_video(path)
        videos_summaries.append(
            {
                "video_category": category_name,
                'sub_category': sub_cat,
                "video_name": video,
                "summary": cap_vid
            }
        )

with open(output_file_labels, "w") as file:
    json.dump(videos_summaries, file, indent=4)

#### 3- Difficult Category

In [ ]:
category_name = "3-DifficultCategory"
category_path = "../DatasetDivided" + "/" + category_name

In [ ]:
sub_cat_mc_path = os.listdir(category_path)
print(sub_cat_mc_path)

In [ ]:
if not os.path.exists("labels"):
    os.makedirs("labels")
# Save the updated list to a JSON file
output_file_labels = "labels/dc_videos_labels.json"

In [ ]:
videos_summaries = []
for sub_cat in sub_cat_mc_path:
    sub_cat_path = category_path + '/' + sub_cat
    videos = os.listdir(sub_cat_path)
    for video in videos:
        path = sub_cat_path + "/" + video
        cap_vid = caption_video(path)
        videos_summaries.append(
            {
                "video_category": category_name,
                'sub_category': sub_cat,
                "video_name": video,
                "summary": cap_vid
            }
        )

with open(output_file_labels, "w") as file:
    json.dump(videos_summaries, file, indent=4)